[![deep-learning-notes](https://github.com/semilleroCV/deep-learning-notes/raw/main/assets/banner-notebook.png)](https://github.com/semilleroCV/deep-learning-notes)

**NOTE:** List bellow is just a placeholder. Feel free to add, remove and swap points, if you need.

- Setup: Install necessary dependencies or clone required repositories.
- Documentation: Provide clear explanations of code, methodology, and results to facilitate understanding for future users.
- References: Acknowledge sources of code, papers, data, and techniques used in the notebook.

## **DenseNet from scratch**

In [4]:
#@title **Install required packages**
! pip install torchinfo


[notice] A new release of pip is available: 23.1.2 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
#@title **Importing libraries**

import torch # 2.3.1+cu121
import torchinfo # 1.8.0

import torch.nn as nn
from torch import Tensor

In [6]:
# Note: Not all dependencies have the __version__ method.

print(f"torch version: {torch.__version__}")
print(f"torchinfo version: {torchinfo.__version__}")

torch version: 2.3.1+cu121
torchinfo version: 1.8.0


**DenseNet (Dense Convolutional Network)** : (brief explanation soon)

**DenseNet architecture code**

In [14]:
class DenseLayer(nn.Module):
  def __init__(self, in_channels: int, growth_rate: int):
    super(DenseLayer, self).__init__()

    self.bn1 = nn.BatchNorm2d(in_channels)
    self.conv1 = nn.Conv2d(in_channels, 4 * growth_rate, kernel_size=1, bias=False)

    self.bn2 = nn.BatchNorm2d(4*growth_rate)
    self.conv2 = nn.Conv2d(4*growth_rate, growth_rate, kernel_size=3, padding=1, bias=False)

    self.relu = nn.ReLU()

  def forward(self, x: Tensor):
    x_in = x

    x = self.bn1(x)
    x = self.relu(x)
    x = self.conv1(x)

    x = self.bn2(x)
    x = self.relu(x)
    x = self.conv2(x)

    return torch.cat([x_in,x], dim=1)


class TransitionLayer(nn.Module):
  def __init__(self, in_channels: int, out_channels: int):
    super(TransitionLayer, self).__init__()

    self.bn = nn.BatchNorm2d(in_channels)
    self.relu = nn.ReLU()
    self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False)
    self.pool = nn.AvgPool2d(kernel_size=2, stride=2)

  def forward(self, x: Tensor):
    x = self.bn(x)
    x = self.relu(x)
    x = self.conv(x)
    x = self.pool(x)

    return x


class DenseBlock(nn.Module):
  def __init__(self, in_channels: int, num_layers: int, growth_rate: int):
    super(DenseBlock, self).__init__()

    self.block = nn.Sequential(*[DenseLayer(in_channels+idx*growth_rate, growth_rate) for idx in range(num_layers)])

  def forward(self, x: Tensor):
    return self.block(x)


class ClassificationHead(nn.Module):
  def __init__(self, in_channels: int, num_classes: int):
    super(ClassificationHead, self).__init__()

    self.pool = nn.AdaptiveAvgPool2d((1,1))
    self.fc = nn.Linear(in_channels, num_classes)

  def forward(self, x: Tensor):

    x = self.pool(x)
    x = torch.flatten(x, start_dim=1)
    x = self.fc(x)

    return x


class DenseNet(nn.Module):
  def __init__(self, layers_per_block: list, in_channels: int, init_output_channels: int, growth_rate: int, num_classes: int):
    super(DenseNet, self).__init__()

    self.conv_pool = nn.Sequential(
        nn.Conv2d(in_channels, init_output_channels, kernel_size=7, stride=2, padding=3, bias=False),
        nn.BatchNorm2d(init_output_channels),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
    )

    self.blocks = nn.ModuleList([])

    blocks_in_channels = init_output_channels
    for i, n_layers in enumerate(layers_per_block):
        dense_block = DenseBlock(blocks_in_channels, n_layers, growth_rate)
        self.blocks.append(dense_block)
        blocks_in_channels += growth_rate * n_layers

        if i != len(layers_per_block) - 1:
            transition_layer = TransitionLayer(blocks_in_channels, blocks_in_channels // 2)
            self.blocks.append(transition_layer)
            blocks_in_channels = blocks_in_channels // 2

    self.bn = nn.BatchNorm2d(blocks_in_channels)
    self.classifier = ClassificationHead(blocks_in_channels, num_classes)

  def forward(self, x: Tensor):
    x = self.conv_pool(x)
    x = nn.ReLU()(x)
    for block in self.blocks:
      x = block(x)

    x = self.classifier(x)

    return x

In [19]:
densenet_config = {
    'densenet-121': [6,12,24,16],
    'densenet-169': [6,12,32,32],
    'densenet-201': [6,12,48,32],
    'densenet-264': [6,12,64,48]
}

model = DenseNet(layers_per_block=densenet_config['densenet-121'], in_channels=3, init_output_channels=64, growth_rate=32, num_classes=1000)
torchinfo.summary(model, (3, 224, 224), batch_dim = 0)

Layer (type:depth-idx)                        Output Shape              Param #
DenseNet                                      [1, 1000]                 2,048
├─Sequential: 1-1                             [1, 64, 56, 56]           --
│    └─Conv2d: 2-1                            [1, 64, 112, 112]         9,408
│    └─BatchNorm2d: 2-2                       [1, 64, 112, 112]         128
│    └─ReLU: 2-3                              [1, 64, 112, 112]         --
│    └─MaxPool2d: 2-4                         [1, 64, 56, 56]           --
├─ModuleList: 1-2                             --                        --
│    └─DenseBlock: 2-5                        [1, 256, 56, 56]          --
│    │    └─Sequential: 3-1                   [1, 256, 56, 56]          335,040
│    └─TransitionLayer: 2-6                   [1, 128, 28, 28]          --
│    │    └─BatchNorm2d: 3-2                  [1, 256, 56, 56]          512
│    │    └─ReLU: 3-3                         [1, 256, 56, 56]          --
│    │ 